In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
dir= r'C:\Users\Admin\Desktop\DPL302m\RockPapperScissor\img'
df= pd.read_csv(r'C:\Users\Admin\Desktop\DPL302m\RockPapperScissor\img\RPS_data.csv')
# encoder= lambda x: tf.one_hot(x, depth=4)
# df['label']=df['label'].apply(encoder)

train_datagen= tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
)
test_datagen= tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)



In [3]:
df= df.sample(frac=1)
batch_size=8
img_num= df.shape[0]
batch_split= int(np.ceil(df.shape[0]/batch_size)*0.8)
split= batch_size*batch_split

train=df.iloc[:split,:]
test= df.iloc[split:,:]


In [4]:
data= train_datagen.flow_from_dataframe(train, x_col='path', y_col='label', class_mode='raw', target_size=(256,256), batch_size=batch_size)
test_data= test_datagen.flow_from_dataframe(test, x_col='path', y_col='label', class_mode='raw', target_size=(256,256), batch_size=batch_size)


Found 720 validated image filenames.
Found 180 validated image filenames.


In [5]:
def split_dataIter(dataIter):
    x=[]
    y=[]
    temp= len(dataIter)
    if len(dataIter[temp-1][0]) != len(dataIter[temp-2][0]):
        temp= temp-1
                                        
    for i in range(temp):
        x.append(dataIter[i][0])
        y.append(dataIter[i][1])
    
    return (np.array(x),np.array(y))

In [6]:
len(test_data[5])

2

In [7]:
x_train, y_train= split_dataIter(dataIter=data)
x_test, y_test= split_dataIter(dataIter=test_data)

y_train=np.array([tf.one_hot(i, depth=4) for i in y_train])
y_test= np.array([tf.one_hot(i, depth=4) for i in y_test])


In [8]:
x_test.shape

(22, 8, 256, 256, 3)

In [9]:
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

In [10]:
def make_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(256,256,3), batch_size= batch_size),
    tf.keras.layers.Conv2D(64,3),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64,3),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128,3),
    tf.keras.layers.MaxPool2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Normalization(),
    tf.keras.layers.Dense(512, activation=tf.keras.activations.elu, activity_regularizer='L2'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu, activity_regularizer='L2'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Normalization(),
    tf.keras.layers.Dense(4, activation=tf.keras.activations.softmax)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=10e-5), 
        loss=tf.keras.losses.categorical_crossentropy,
        metrics='accuracy'
    )
    
    return model

model= make_model()


In [11]:
for k in range(10):
     for i in range(batch_split):
          model.fit(x_train[i], y_train[i], batch_size=batch_size, epochs=1, shuffle=True)

1/1 [==============================] - 0s 72ms/step - loss: 0.2237 - accuracy: 1.0000


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential/dense/MatMul/MatMul_1' defined at (most recent call last):
    File "c:\Users\Admin\.conda\envs\tfGPU\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Admin\.conda\envs\tfGPU\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Admin\.conda\envs\tfGPU\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Admin\.conda\envs\tfGPU\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\Admin\.conda\envs\tfGPU\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Admin\.conda\envs\tfGPU\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Admin\.conda\envs\tf

In [ ]:
for i in range(11):
    model.evaluate(x_test[i], y_test[i], batch_size=batch_size)